In [0]:
from os import path
import sqlite3


In [0]:
connection = sqlite3.connect('iam_words.db')
cursor = connection.cursor()

In [0]:
cursor.execute('DROP TABLE IF EXISTS words_index')
cursor.execute('''CREATE TABLE words_index (
                    word_id TEXT,
                    image_seq_path TEXT,
                    image_path TEXT,                    
                    status TEXT)''')
                    #gray_level INTEGER,
                    #tokens TEXT

In [0]:
with open('../iam-database_WordSeq/wordList.txt', 'r') as handle:
    for line in handle:
        if line.startswith('#'):
            # In the data-file, lines starting with
            # the '#' symbol are comments.
            # Ignoring the comments
            continue
        
        # Each row of data has nine parts separated by a single space ' '.
        # The parts are:
        # - word_id
        # - status: result of word segmentation ('ok', 'er')
        # - graylevel: the gray-level value to binarize the image
        # - n_components
        # - x, y, w, h: bounding box around this word
        # - tokens: transcription for this word.  
        parts = line.strip().split(' ')
        print(parts)
        word_id = parts[0]
        #word_id, status, gray_level, n_components, x, y, w, h = parts[:8]
        #tokens = ' '.join(parts[8:])

        # Each word_id is of the format a-b-c-d
        # such that the corresponding image is located at
        # words/a/a-b/a-b-c-d.png
        first, second, third, fourth = word_id.split('-')
        outer = first
        inner = first + '-' + second
        file_name = first + '-' + second + '-' + third + '-' + fourth + '.txt'
        image_name = first + '-' + second + '-' + third + '-' + fourth + '.png'        
        image_seq_path = path.join('sequences', outer, inner, file_name)
        image_path = path.join('words', outer, inner, image_name)        
        values = (word_id, image_seq_path, image_path, parts[1])
        #, 
        #   int(gray_level), tokens)
        
        cursor.execute('INSERT INTO words_index VALUES(?, ?, ?,?)', values)

result = next(cursor.execute('SELECT COUNT(*) from words_index'))
print ('Inserted {:,d} rows.'.format(result[0]))

for row in cursor.execute('SELECT * FROM words_index WHERE status ="front" '):
    print(row)

for row in cursor.execute('SELECT count (distinct status) FROM words_index'):
    print('No. of words (distinct):', row)



In [0]:
connection.commit()
connection.close()